In [ ]:
let input =
    File.ReadAllLines @"input/11.txt"
    |> Array.map (fun line -> 
        line 
        |> Seq.map (fun c -> c |> string |> int)
        |> Array.ofSeq)
input

index,value
0,"[ 5, 4, 8, 3, 1, 4, 3, 2, 2, 3 ]"
1,"[ 2, 7, 4, 5, 8, 5, 4, 7, 1, 1 ]"
2,"[ 5, 2, 6, 4, 5, 5, 6, 1, 7, 3 ]"
3,"[ 6, 1, 4, 1, 3, 3, 6, 1, 4, 6 ]"
4,"[ 6, 3, 5, 7, 3, 8, 5, 4, 7, 8 ]"
5,"[ 4, 1, 6, 7, 5, 2, 4, 6, 4, 5 ]"
6,"[ 2, 1, 7, 6, 8, 4, 1, 7, 2, 1 ]"
7,"[ 6, 8, 8, 2, 8, 8, 1, 1, 3, 4 ]"
8,"[ 4, 8, 4, 6, 8, 4, 8, 5, 5, 4 ]"
9,"[ 5, 2, 8, 3, 7, 5, 1, 5, 2, 6 ]"


In [ ]:
let h = input.Length
let w = input[0].Length

let neighbors (x, y) =
  [ (x - 1, y - 1)
    (x - 1, y)
    (x - 1, y + 1)
    (x, y - 1)
    (x, y + 1)
    (x + 1, y - 1)
    (x + 1, y)
    (x + 1, y + 1) ]
    |> List.filter (fun (x',y') -> x' >= 0 && x' < w && y' >= 0 && y' < h)

let rec afterFlash (visited: list<int*int>) (board:int array array) =
    match visited with
    | [] -> board
    | _ ->
        let currentPos = List.head visited
        let cx,cy = currentPos
        let FLASHED = -10
        let newVal = board.[cy].[cx] + 1
        if newVal > 9 then
            // printfn "%d,%d" cx cy
            let newBoard = 
                board 
                |> Array.updateAt cy 
                        (Array.updateAt cx FLASHED board.[cy])
            let newVisited = (List.tail visited) @ neighbors currentPos
            afterFlash newVisited newBoard
        else
            let newBoard = 
                board 
                |> Array.updateAt cy 
                        (Array.updateAt cx newVal board.[cy])
            afterFlash (List.tail visited) newBoard

In [ ]:
#!time
// Part A
// non-optimal solution
// scan will create all board within range first then calc sum
[0..99]
|> List.scan 
    (fun board _ -> 
        board 
        |> afterFlash [for x in 0..w-1 do for y in 0..h-1 -> (x,y)]
        |> Array.map (fun row -> row |> Array.map (fun k -> if k < 0 || k > 9 then 0 else k))
        ) input
|> List.map (fun board -> 
                board
                |> Array.map(fun row -> row |> Array.filter (fun k -> k = 0) |> Array.length)
                |> Array.sum)
|> List.sum


1656

Wall time: 34.0855ms

In [ ]:
#!time
// Part B
let rec findFirstAllFlash board count = 
    let flashCount = 
        board
        |> Array.map(fun row -> row |> Array.filter (fun k -> k = 0) |> Array.length)
        |> Array.sum

    if flashCount = w*h then
        count
    else 
        let newBoard = 
            board 
            |> afterFlash [for x in 0..w-1 do for y in 0..h-1 -> (x,y)]
            |> Array.map (fun row -> row |> Array.map (fun k -> if k < 0 || k > 9 then 0 else k))
        findFirstAllFlash newBoard (count+1)

findFirstAllFlash input 0

195

Wall time: 37.7054ms